In [50]:
from classiq import QBit, Output, qfunc, RX, RZ, allocate
from sympy import pi

# Define the quantum circuit for the harmonic oscillator
@qfunc
def main(y1: Output[QBit], y2: Output[QBit]):
    # Allocate qubits for y1 and y2
    allocate(1, y1)
    allocate(1, y2)

    # Initialize the qubits to represent the initial state y(0) = 1 and y'(0) = 1
    RX(pi, y1)  # Apply RX(pi) to y1 to set y1 = 1
    RX(pi, y2)  # Apply RX(pi) to y2 to set y2 = 1

# Synthesize the quantum model
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

# Execute the quantum program
job = execute(quantum_program)

# Fetch and print the results
results = job.result()[0].value
print("Parsed counts:", results.parsed_counts)
print("Parsed states:", results.parsed_states)

Parsed counts: [{'y1': 1.0, 'y2': 1.0}: 2048]
Parsed states: {'11': {'y1': 1.0, 'y2': 1.0}}


In [51]:
# Matrix M is already defined as M = [[0, 1], [-1, 0]]
@qfunc
def time_evolution(y1: QBit, y2: QBit):
    # Apply time evolution gates based on matrix M
    RZ(pi / 2, y1)  # Evolves y1
    RX(pi / 2, y2)  # Evolves y2

# Include the time evolution step in the circuit

In [52]:
# Define the quantum circuit for time evolution
@qfunc
def time_evolution(y1: QBit, y2: QBit):
    # Time evolution step (applying matrix M)
    RX(pi / 2, y1)  # Evolve y1 based on M
    RZ(pi / 2, y2)  # Evolve y2 based on M

# Now integrate the time evolution into the main function
@qfunc
def main(y1: Output[QBit], y2: Output[QBit]):
    # Allocate qubits for y1 and y2
    allocate(1, y1)
    allocate(1, y2)

    # Initialize the qubits to represent the initial state y(0) = 1 and y'(0) = 1
    RX(pi, y1)  # Apply RX(pi) to y1 to set y1 = 1
    RX(pi, y2)  # Apply RX(pi) to y2 to set y2 = 1

    # Apply time evolution using the matrix M
    time_evolution(y1, y2)

# Synthesize and execute the quantum model
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

# Execute the quantum program
job = execute(quantum_program)

job.open_in_ide()

# Fetch and print the results
results = job.result()[0].value
print("Parsed counts:", results.parsed_counts)
print("Parsed states:", results.parsed_states)

Parsed counts: [{'y1': 1.0, 'y2': 1.0}: 1030, {'y1': 0.0, 'y2': 1.0}: 1018]
Parsed states: {'10': {'y1': 0.0, 'y2': 1.0}, '11': {'y1': 1.0, 'y2': 1.0}}


In [53]:
# Loop through the parsed counts to calculate energies based on measured y1 and y2
for outcome in results.parsed_counts:
    y1 = outcome.state['y1']
    y2 = outcome.state['y2']
    shots = outcome.shots

    # Kinetic energy
    kinetic_energy = 0.5 * y2 ** 2
    potential_energy = 0.5 * y1 ** 2

    total_kinetic_energy += kinetic_energy * shots
    total_potential_energy += potential_energy * shots
    total_shots += shots
    
    # Print for debugging each iteration
    print(f"y1: {y1}, y2: {y2}, shots: {shots}")
    print(f"Kinetic Energy: {kinetic_energy}, Potential Energy: {potential_energy}")

# Average the energies
average_kinetic_energy = total_kinetic_energy / total_shots
average_potential_energy = total_potential_energy / total_shots

# Print the results
print(f"Average Kinetic Energy (KE): {average_kinetic_energy}")
print(f"Average Potential Energy (PE): {average_potential_energy}")

y1: 1.0, y2: 1.0, shots: 1030
Kinetic Energy: 0.5, Potential Energy: 0.5
y1: 0.0, y2: 1.0, shots: 1018
Kinetic Energy: 0.5, Potential Energy: 0.0
Average Kinetic Energy (KE): 0.5
Average Potential Energy (PE): 0.2511393229166667
